In [21]:
import pandas as pd
import matplotlib.pyplot as plt
import re

# Load the dataset
series = pd.read_csv("series-dataset-2023.csv")

# Rename the column for consistency
series.rename(columns={'anime_id': 'id'}, inplace=True)

# Function to clean titles
def clean_title(title):
    return re.sub(r"[^a-zA-Z0-9 ]", "", title)

# Apply the cleaning function to the 'English name' column
series['clean_title'] = series['English name'].apply(clean_title)

# Drop rows with missing values in specified columns
series_cleaned = series.dropna(subset=['Name', 'clean_title', 'Score', 'id'])

# Print the cleaned data
print(series_cleaned[['Name', 'clean_title']].head())


                              Name             clean_title
0                     Cowboy Bebop            Cowboy Bebop
1  Cowboy Bebop: Tengoku no Tobira  Cowboy Bebop The Movie
2                           Trigun                  Trigun
3               Witch Hunter Robin      Witch Hunter Robin
4                   Bouken Ou Beet  Beet the Vandel Buster


In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(ngram_range=(1,2))

tfidf = vectorizer.fit_transform(series["clean_title"])


In [23]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np


def search(name):
    # Ensure 'series' DataFrame has been defined and contains the necessary columns
    if 'series' not in globals():
        print("The 'series' DataFrame is not defined.")
        return pd.DataFrame()

    # Check for the correct column names
    if 'Name' not in series.columns or 'id' not in series.columns or 'English name' not in series.columns:
        print("The 'series' DataFrame does not contain the required columns.")
        return pd.DataFrame()

    # Perform the search with case-insensitivity and handling NaN values
    results = series[series['Name'].str.contains(name, case=False, na=False)]

    # If no results are found, return an empty DataFrame
    if results.empty:
        print("No results found.")
        return pd.DataFrame()  # Return an empty DataFrame if no matches are found

    # Ensure results are sorted or handled as needed
    return results[['id', 'English name']]


In [26]:
import ipywidgets as widgets
from IPython.display import display

series_input = widgets.Text(
    value='',
    description='Series Name:',
    disabled=False
)

series_list = widgets.Output()

def on_type(data):
    with series_list:
        series_list.clear_output()
        name = data["new"]
        if len(name) > 0:  # Check if the input length is sufficient
            results = search(name)
            if not results.empty:
                # Display the results with English names
                display(results)

series_input.observe(on_type, names='value')

display(series_input, series_list)

Text(value='', description='Series Name:')

Output()

In [31]:

ratings = pd.read_csv("rating.csv")
seriesID = 21
ratings.rename(columns={'anime_id': 'id'}, inplace=True)
seriess = series[series["id"] == seriesID]
ratings.head()


,user_id,id,rating
0,1,20.0,-1.0
1,1,24.0,-1.0
2,1,79.0,-1.0
3,1,226.0,-1.0
4,1,241.0,-1.0


In [32]:
similar_users = ratings[(ratings["id"] == seriesID) & (ratings["rating"] > 7)]["user_id"].unique()
similar_user_recs = ratings[(ratings["user_id"].isin(similar_users)) & (ratings["rating"] > 7)]["id"]
similar_user_recs = similar_user_recs.value_counts() / len(similar_users)
similar_user_recs = similar_user_recs[similar_user_recs > 0.1]



In [33]:
all_users = ratings[(ratings["id"].isin(similar_user_recs.index)) & (ratings["rating"] > 7)]
all_user_recs = all_users["id"].value_counts() / len(all_users["user_id"].unique())

In [34]:
rec_percentages = pd.concat([similar_user_recs, all_user_recs], axis=1)
rec_percentages.columns = ["similar", "all"]



In [35]:
rec_percentages["score"] = rec_percentages["similar"] / rec_percentages["all"]


rec_percentages = rec_percentages.sort_values("score", ascending=False)

rec_percentages.head(10).merge(series, left_index=True, right_on="id")



,similar,all,score,id,Name,English name,Other name,Score,Genres,Synopsis,...,Source,Duration,Rating,Rank,Popularity,Favorites,Scored By,Members,Image URL,clean_title


In [36]:
def find_similar_series(id):
    similar_users = ratings[(ratings["id"] == id) & (ratings["rating"] > 4)]["user_id"].unique()
    similar_user_recs = ratings[(ratings["user_id"].isin(similar_users)) & (ratings["rating"] > 4)]["id"]
    similar_user_recs = similar_user_recs.value_counts() / len(similar_users)
    similar_user_recs = similar_user_recs[similar_user_recs > .10]

    all_users = ratings[(ratings["id"].isin(similar_user_recs.index)) & (ratings["rating"] > 4)]
    all_user_recs = all_users["id"].value_counts() / len(all_users["user_id"].unique())
    rec_percentages = pd.concat([similar_user_recs, all_user_recs], axis=1)
    rec_percentages.columns = ["similar", "all"]

    rec_percentages["score"] = rec_percentages["similar"] / rec_percentages["all"]
    rec_percentages = rec_percentages.sort_values("similar", ascending=False)
    return rec_percentages.head(10).merge(series, left_index=True, right_on="id")


In [37]:
import ipywidgets as widgets
from IPython.display import display

movie_name_input = widgets.Text(
    value='',
    description='Show :',
    disabled=False
)
recommendation_list = widgets.Output()

def on_type(data):
    with recommendation_list:
        recommendation_list.clear_output()
        name = data["new"]
        if len(name) > 5:
            results = search(name)
            seriesID = results.iloc[0]["id"]
            display(find_similar_series(seriesID))

movie_name_input.observe(on_type, names='value')

display(movie_name_input, recommendation_list)


Text(value='', description='Show :')

Output()